# Bike Sharing Demand Prediction

## Problem Statement

A bike-sharing system is a service in which bikes are made available for shared use to individuals on a short term basis for a price or free. Many bike share systems allow people to borrow a bike from a "dock" which is usually computer-controlled wherein the user enters the payment information, and the system unlocks it. This bike can then be returned to another dock belonging to the same system. <br>

A US bike-sharing provider BoomBikes has recently suffered considerable dips in their revenues due to the ongoing Corona pandemic. The company is finding it very difficult to sustain in the current market scenario. So, it has decided to come up with a mindful business plan to be able to accelerate its revenue as soon as the ongoing lockdown comes to an end, and the economy restores to a healthy state. 

In such an attempt, BoomBikes aspires to understand the demand for shared bikes among the people after this ongoing quarantine situation ends across the nation due to Covid-19. They have planned this to prepare themselves to cater to the people's needs once the situation gets better all around and stand out from other service providers and make huge profits.

They have contracted a consulting company to understand the factors on which the demand for these shared bikes depends. Specifically, they want to understand the factors affecting the demand for these shared bikes in the American market. The company wants to know:

- Which variables are significant in predicting the demand for shared bikes.
- How well those variables describe the bike demands

Based on various meteorological surveys and people's styles, the service provider firm has gathered a large dataset on daily bike demands across the American market based on some 
factors. 

## Business Goal

Model the demand for shared bikes with the available independent variables. <br>
It will be used by the management to understand how exactly the demands vary with different features.



### Libraries

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

### Load Dataset and Data Dictionary

In [2]:
# Load Dataset
bikes_df = pd.read_csv('day.csv')
bikes_df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,1,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,1,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,1,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,1,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,1,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


In [3]:
# Load Dictionary

with open('Readme.txt') as file:
    print(file.read())

Dataset characteristics
day.csv have the following fields:
	
	- instant: record index
	- dteday : date
	- season : season (1:spring, 2:summer, 3:fall, 4:winter)
	- yr : year (0: 2018, 1:2019)
	- mnth : month ( 1 to 12)
	- holiday : weather day is a holiday or not (extracted from http://dchr.dc.gov/page/holiday-schedule)
	- weekday : day of the week
	- workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
	+ weathersit : 
		- 1: Clear, Few clouds, Partly cloudy, Partly cloudy
		- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
		- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
		- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
	- temp : temperature in Celsius
	- atemp: feeling temperature in Celsius
	- hum: humidity
	- windspeed: wind speed
	- casual: count of casual users
	- registered: count of registered users
	- cnt: count of total rental bikes including both casual and registered
	
License

On understanding the dataset and the meaning of the features, we can intially drop a few features from our analysis as 
they will not add any value to the model building. <br>

1.  __instant__: Can be removed as it is just the record index.
2.  __casual__ and __registered__: Can be removed as they are jointly the Target Variabel cnt 

In [4]:
bikes_df.drop(columns=['instant','casual','registered'],inplace=True)

In [5]:
bikes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   dteday      730 non-null    object 
 1   season      730 non-null    int64  
 2   yr          730 non-null    int64  
 3   mnth        730 non-null    int64  
 4   holiday     730 non-null    int64  
 5   weekday     730 non-null    int64  
 6   workingday  730 non-null    int64  
 7   weathersit  730 non-null    int64  
 8   temp        730 non-null    float64
 9   atemp       730 non-null    float64
 10  hum         730 non-null    float64
 11  windspeed   730 non-null    float64
 12  cnt         730 non-null    int64  
dtypes: float64(4), int64(8), object(1)
memory usage: 74.3+ KB


It appears to be the case that there are no missing values in the dataset.

### Descriptive Statistics

In [6]:
bikes_df.describe()

,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
count,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000
mean,2.498630,0.500000,6.526027,0.028767,2.997260,0.683562,1.394521,20.319259,23.726322,62.765175,12.763620,4508.006849
std,1.110184,0.500343,3.450215,0.167266,2.006161,0.465405,0.544807,7.506729,8.150308,14.237589,5.195841,1936.011647
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,2.424346,3.953480,0.000000,1.500244,22.000000
25%,2.000000,0.000000,4.000000,0.000000,1.000000,0.000000,1.000000,13.811885,16.889713,52.000000,9.041650,3169.750000
50%,3.000000,0.500000,7.000000,0.000000,3.000000,1.000000,1.000000,20.465826,24.368225,62.625000,12.125325,4548.500000
75%,3.000000,1.000000,10.000000,0.000000,5.000000,1.000000,2.000000,26.880615,30.445775,72.989575,15.625589,5966.000000
max,4.000000,1.000000,12.000000,1.000000,6.000000,1.000000,3.000000,35.328347,42.044800,97.250000,34.000021,8714.000000


In [7]:
bikes_df.dtypes

dteday         object
season          int64
yr              int64
mnth            int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
cnt             int64
dtype: object

In [8]:
# Extract day of the month from Dates
bikes_df['dates'] = pd.to_datetime(bikes_df.dteday,format = '%d-%m-%Y').dt.day

In [9]:
# drop dteday
bikes_df.drop(columns=['dteday'],inplace=True)

In [10]:
bikes_df.head()

,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt,dates
0,1,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,985,1
1,1,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,801,2
2,1,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,1349,3
3,1,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,1562,4
4,1,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,1600,5


Now the continous features as per the dictionary are __temp, atemp, hum, windspeed__. <br/>
And as seen from above, we do not need to make any changes to the data types for these columns.

### Handling Categorical Features

Before proceeding further we have to handle the categorical features.

1.  Replace values in categorical columns with more interpretable values rather than numerical values.
2.  Convert data type of dates column to categorical.
3.  One-Hot Encode the categorical variables.

In [11]:
bikes_df['season'] = bikes_df.season.map({1:'spring', 2:'summer', 3:'fall', 4:'winter'})

In [12]:
bikes_df['mnth'] = bikes_df.mnth.map({1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}
)

In [13]:
bikes_df['weekday'] = bikes_df.weekday.map({0:'sunday',1:'monday',2:'tuesday',3:'wednesday',4:'thursday',5:'friday',6:'saturday'})

In [14]:
bikes_df['holiday'] = bikes_df.holiday.map({0:'not_holiday',1:'holiday'})

In [15]:
bikes_df['workingday'] = bikes_df.workingday.map({0:'non_working_day',1:'working_day'})

In [16]:
# Lets decode the weathersit as follows:
# 1. Clear, Few clouds, Partly cloudy, Partly cloudy : almost_clear
# 2. Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist : misty
# 3. Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds : lighy_rain_snow
# 4. Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog: extreme


bikes_df['weathersit'] = bikes_df.weathersit.map({1: 'almost_clear', 2: 'misty', 3: 'lighy_rain_snow',4: 'extreme'})

In [17]:
bikes_df.head()

,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt,dates
0,spring,0,Jan,not_holiday,saturday,non_working_day,misty,14.110847,18.18125,80.5833,10.749882,985,1
1,spring,0,Jan,not_holiday,sunday,non_working_day,misty,14.902598,17.68695,69.6087,16.652113,801,2
2,spring,0,Jan,not_holiday,monday,working_day,almost_clear,8.050924,9.47025,43.7273,16.636703,1349,3
3,spring,0,Jan,not_holiday,tuesday,working_day,almost_clear,8.200000,10.60610,59.0435,10.739832,1562,4
4,spring,0,Jan,not_holiday,wednesday,working_day,almost_clear,9.305237,11.46350,43.6957,12.522300,1600,5


In [22]:
bikes_df['dates'] = bikes_df.dates.astype('category')

In [23]:
bikes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   season      730 non-null    object  
 1   yr          730 non-null    int64   
 2   mnth        730 non-null    object  
 3   holiday     730 non-null    object  
 4   weekday     730 non-null    object  
 5   workingday  730 non-null    object  
 6   weathersit  730 non-null    object  
 7   temp        730 non-null    float64 
 8   atemp       730 non-null    float64 
 9   hum         730 non-null    float64 
 10  windspeed   730 non-null    float64 
 11  cnt         730 non-null    int64   
 12  dates       730 non-null    category
dtypes: category(1), float64(4), int64(2), object(6)
memory usage: 70.8+ KB


We have mapped all categories respectively, now we will convert them into one-hot encoded format.